In [1]:
VERSION = '99_v1_04_01'

In [2]:
model_list = [    
    #{'path':'../../13_Baseline5/exp/result/13_v1_03/oof_df.csv', 'w':1}, #deberta-v3-base, seed100, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_04/oof_df.csv', 'w':1}, #deberta-v3-base, seed200, split00_v2_07
    #{'path':'../../13_Baseline5/exp/result/13_v1_07/oof_df.csv', 'w':1}, #deberta-base, seed100, split00_v2_07
    
    {'path':'../../18_BaselineSW/exp/result/18_v2_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_03/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v2_04/oof_df.csv', 'w':1}, #longformer-large, seed100, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v2_07/oof_df.csv', 'w':1}, #deberta-large, seed200, split00_v2_07
    #{'path':'../../18_BaselineSW/exp/result/18_v1_04/oof_df.csv', 'w':1}, #deberta-v3-large, seed200, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v1_05/oof_df.csv', 'w':1}, #deberta-xlarge, seed100, split00_v2_07
    {'path':'../../18_BaselineSW/exp/result/18_v2_22/oof_df.csv', 'w':1}, #deberta-v2-xlarge, seed100, split00_v2_07
    
    #{'path':'../../19_RNN/exp/result/19_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, lstm
    
    {'path':'../../20_WoSpanDet/exp/result/20_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, huggingface pre
    #{'path':'../../20_WoSpanDet/exp/result/20_v1_02/oof_df.csv', 'w':1}, #deberta-v3-large, seed100, split00_v2_07, huggingface pre
    
    {'path':'../../22_BaselineMLM/exp/result/22_v1_01/oof_df.csv', 'w':1}, #deberta-large, seed100, split00_v2_07, MLM pre
    {'path':'../../22_BaselineMLM/exp/result/22_v2_01/oof_df.csv', 'w':1}, #deberta-xlarge, seed100, split00_v2_07, MLM pre
    
    #{'path':'../../999_Stacking/999_v1_11/result/oof_cat.csv', 'w':1}, #stacking (base x2 + longformer)
    #{'path':'../../999_Stacking/999_v1_13/result/oof_cat.csv', 'w':1}, #stacking (base x2)
]

In [3]:
import pandas as pd

w_list = []
for i,model in enumerate(model_list):
    w = model['w']
    w_list.append(w)

stack_df = None
for i,model in enumerate(model_list):
    pred_df = pd.read_csv(model['path'])
    w = w_list[i]
    
    if 'cat' in model['path']:
        pred_df['pred_ineffective'] = pred_df['oof_ineffective'] * w / sum(w_list)
        pred_df['pred_adequate'] = pred_df['oof_adequate'] * w / sum(w_list)
        pred_df['pred_effective'] = pred_df['oof_effective'] * w / sum(w_list)
    else:
        pred_df['pred_ineffective'] = pred_df['pred_ineffective'] * w / sum(w_list)
        pred_df['pred_adequate'] = pred_df['pred_adequate'] * w / sum(w_list)
        pred_df['pred_effective'] = pred_df['pred_effective'] * w / sum(w_list)
    pred_df = pred_df.rename(columns={
        'pred_ineffective':f'Ineffective_{i}', 
        'pred_adequate':f'Adequate_{i}', 
        'pred_effective':f'Effective_{i}'
    })
    if stack_df is None:
        stack_df = pred_df.copy()
    else:
        stack_df = stack_df.merge(pred_df[['discourse_id',f'Ineffective_{i}',f'Adequate_{i}',f'Effective_{i}']],
                                  on='discourse_id', how='left')
    

train_df = pd.read_csv('../../input/feedback-prize-effectiveness/train.csv')
oof_df = train_df[['discourse_id']].merge(stack_df, on='discourse_id', how='left')

In [4]:
for col_name in ['Ineffective','Adequate','Effective']:
    cols = [f'{col_name}_{i}' for i in range(len(model_list))]
    oof_df[col_name] = oof_df[cols].sum(axis=1)

In [5]:
from sklearn.metrics import log_loss

score = log_loss(oof_df['label'].values, oof_df[['Ineffective','Adequate','Effective']].values)
print('oof={:.4f}'.format(score))

oof=0.5797


In [6]:
import os
os.makedirs('result', exist_ok=True)

oof_df.to_csv(f'result/stack_{VERSION}.csv', index=False)

In [7]:
oof_df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,Ineffective_0,Adequate_0,Effective_0,label,loss,...,Effective_4,Ineffective_5,Adequate_5,Effective_5,Ineffective_6,Adequate_6,Effective_6,Ineffective,Adequate,Effective
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate,0.001864,0.131125,0.009868,1,0.085691,...,0.003289,0.011671,0.129193,0.001993,0.006413,0.133635,0.002809,0.044823,0.922032,0.033145
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a ...",Position,Adequate,0.007999,0.130910,0.003947,1,0.087332,...,0.002593,0.022810,0.117033,0.003014,0.010032,0.129234,0.003592,0.088844,0.884978,0.026178
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform be...,Claim,Adequate,0.020101,0.121164,0.001592,1,0.164698,...,0.001091,0.041657,0.099220,0.001980,0.015242,0.124866,0.002749,0.162900,0.821011,0.016089
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The...",Evidence,Adequate,0.047771,0.093853,0.001233,1,0.420119,...,0.001222,0.048549,0.093130,0.001179,0.044911,0.096469,0.001476,0.247820,0.741526,0.010655
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by ali...,Counterclaim,Adequate,0.035391,0.106863,0.000603,1,0.290297,...,0.000905,0.060683,0.081371,0.000803,0.035708,0.104534,0.002615,0.307772,0.683263,0.008965
